# Full separability permutation criteria for two-qubit states

## Context

In this notebook we apply the 2 independent two-qubit separability permutation criteria to both states and projective measurement operators. The permutation criteria are given in the article https://doi.org/10.48550/arXiv.quant-ph/0504160 for states. We use the same criteria for projective measurement operators by taking into account the fact that any projection $\Pi$ of the form

$$
\Pi = \sum_{j=1}^N | \phi_j \rangle \langle \phi_j |,
$$

where each $|\phi_j \rangle$ is a normalized state vector, can be associated to a state $\rho$ after normalization:

$$
\rho = \frac{1}{N} \Pi.
$$

When any of the criteria fails (that is any *True* in the following code) for a given state $\rho$, then we can conclude that the state shows some form of entanglement and therefore it is not separable. Since separability does not depend on scaling this allows us to discard the full separability of any corresponding measurement operator. Discarding full separability of a measurement is important since it also discards its implementability by means of LOCC (local operations and classical communication).

## Output

- The information of whether or not we can discard the full separability of any two-qubit state/measurement.

In [1]:
import numpy as np
N = 2 # number of qubits

In [2]:
# some functions

def partial_transpose(A, k): # perform a partial transpose of the k-th qubit in matrix A
    J = list(range(2*N))
    J[2*k] = 2*k+1
    J[2*k+1] = 2*k
    return A.transpose(*J)

def reshuffle(A, k, l):
    J = list(range(2*N))
    J[2*l] = 2*k+1
    J[2*k+1] = 2*l
    return A.transpose(*J)

def statevec_prep(u):
    v = np.zeros([2]*N, dtype='complex')
    for j in range(2**N):
        bits = np.array(list('{0:02b}'.format(j))).astype('int') # from integer to 4 digit binary number
        a, b = bits
        v[a,b] = u[j]
    return v

def statevec_2_operator(v):
    rho = np.zeros([2]*2*N, dtype='complex')
    for j in range(2**N):
        bits1 = np.array(list('{0:02b}'.format(j))).astype('int') # from integer to 4 digit binary number
        a1, b1 = bits1
        for k in range(2**N):
            bits2 = np.array(list('{0:02b}'.format(k))).astype('int') # from integer to 4 digit binary number
            a2, b2 = bits2
            rho[a1,a2,b1,b2] = v[a1,b1]*np.conjugate(v[a2,b2])
    return rho

def rho_2_standard(rho):
    sigma = np.zeros((2**N,2**N), dtype='complex')
    for j in range(2**N):
        bits1 = np.array(list('{0:02b}'.format(j))).astype('int') # from integer to 4 digit binary number
        a1, b1 = bits1
        for k in range(2**N):
            bits2 = np.array(list('{0:02b}'.format(k))).astype('int') # from integer to 4 digit binary number
            a2, b2 = bits2
            sigma[j,k] = rho[a1,a2,b1,b2]
    return sigma

def check_for_neglambda(rho, DEC=15):
    return ((np.round(np.linalg.eigvals(rho_2_standard(rho)), DEC) < 0)).any()

def check_for_tracenorm(rho):
    tracenorm = np.sum(np.abs(np.linalg.eigvals(rho_2_standard(rho))))
    print(tracenorm)
    return tracenorm > 1

def separability_criteria(rho):
    
    # QT Row
    rho_p = partial_transpose(rho,0)
    print("QT \t: ", check_for_tracenorm(rho_p))
        
    # R Row
    rho_p = reshuffle(rho, 1,0)
    print("R \t: ", check_for_tracenorm(rho_p))


# Separable state example

In [3]:
# u_sep will be a separable HD state
u_sep = np.zeros(2**N)
u_sep[0] = 1/np.sqrt(2)
u_sep[1] = 1/np.sqrt(2)
rho = statevec_2_operator(statevec_prep(u_sep))
# print(np.round(np.linalg.eigvals(rho_2_standard(rho)),10)) #check that rho is pure

In [4]:
separability_criteria(rho)
print()
print("If any true, then state rho is NOT fully separable")
print("If all false, then nothing can be stated about rho (generally that it may be separable)")

0.999999999999999
QT 	:  False
0.4999999999999999
R 	:  False

If any true, then state rho is NOT fully separable
If all false, then nothing can be stated about rho (generally that it may be separable)


# Bell state example

In [5]:
# GHZ will be aN HH + VV state
GHZ = np.zeros(2**N)
GHZ[0] = 1/np.sqrt(2)
GHZ[-1] = 1/np.sqrt(2)
rho = statevec_2_operator(statevec_prep(GHZ))
# print(np.round(np.linalg.eigvals(rho_2_standard(rho)),10)) #check that rho is pure

In [6]:
separability_criteria(rho)
print()
print("If any true, then state rho is NOT separable")
print("If all false, then nothing can be stated about rho (generally that it is separable)")

1.9999999999999991
QT 	:  True
1.9999999999999996
R 	:  True

If any true, then state rho is NOT separable
If all false, then nothing can be stated about rho (generally that it is separable)


# D'Espagnat measurement separability test

In [7]:
psi_p = np.zeros(2**N); psi_m = np.zeros(2**N)
psi_p[1] = 1/np.sqrt(2); psi_m[1] = 1/np.sqrt(2)
psi_p[2] = 1/np.sqrt(2); psi_m[2] = -1/np.sqrt(2)

phi_p = np.zeros(2**N); phi_m = np.zeros(2**N)
phi_p[0] = 1/np.sqrt(2); phi_m[0] = 1/np.sqrt(2)
phi_p[3] = 1/np.sqrt(2); phi_m[3] = -1/np.sqrt(2)

rho1 = statevec_2_operator(statevec_prep(psi_p))
rho2 = statevec_2_operator(statevec_prep(psi_m))
rho3 = statevec_2_operator(statevec_prep(phi_p))
rho4 = statevec_2_operator(statevec_prep(phi_m))

rho = (rho1+rho2)/2 # proportional to despagnat measurement operator

In [8]:
separability_criteria(rho)
print()
print("If any true, then state rho is NOT fully separable")
print("If all false, then nothing can be stated about rho (generally that it is separable)")

0.9999999999999998
QT 	:  False
0.9999999999999993
R 	:  False

If any true, then state rho is NOT fully separable
If all false, then nothing can be stated about rho (generally that it is separable)
